In [86]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import numpy as np
from pandas.core.common import flatten
import cv2
import glob
from tqdm import tqdm
from PIL import Image

In [87]:
dataset_paths = 'cropped-rps-dataset'
dataset_image_path = []
classes = []

for data_path in glob.glob(dataset_paths + '/*'):
    classes.append(data_path.split('\\')[-1])
    dataset_image_path.append(glob.glob(data_path + '/*'))

dataset_image_path = list(flatten(dataset_image_path))

print(len(dataset_image_path))

1661


In [88]:
image_transform = transforms.Compose({
    transforms.ToPILImage(),
    transforms.Resize((150,150)),
    transforms.ToTensor(),
   # transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
})

In [89]:
class handDataset(torch.utils.data.Dataset):

    def __init__(self, image_path, transform=None):
        self.image_path = image_path
        self.transform = transform
    
    def __len__(self):
        return len(self.image_path)
    
    def __getitem__(self,idx):
        image_filepath = self.image_path[idx]
        image = cv2.imread(image_filepath)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        label = image_filepath.split('\\')[-2]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

torch_dataset = handDataset(dataset_image_path,image_transform)

In [90]:
batch_size = 10
learnin_rate = 0.0005
num_epochs = 10

In [91]:
generator = torch.Generator().manual_seed(42)

train_size = int(len(torch_dataset) * .7)
test_size = len(torch_dataset) - train_size

train_dataset, test_dataset = torch.utils.data.random_split(torch_dataset, [train_size, test_size], generator=generator)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=True)

In [92]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.hub.load('pytorch/vision:v0.10.0', 'mobilenet_v2', pretrained=True).to(device)

criterion = nn.CrossEntropyLoss()

optimizer = torch.optim.SGD(model.parameters(), lr=learnin_rate, weight_decay=0.005, momentum=0.9)

Using cache found in C:\Users\iii/.cache\torch\hub\pytorch_vision_v0.10.0


In [93]:
for epoch in range(num_epochs):

    for i, (images, labels) in tqdm(enumerate(train_loader)):
        images=images.to(device)
        labels=torch.tensor(labels).to(device)
        
        outputs = model(images)
        loss = criterion(outputs, labels)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    print('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, num_epochs, loss.item()))

0it [00:00, ?it/s]


AttributeError: 'tuple' object has no attribute 'to'